In [1]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import itertools
import random
import time
#import sage

In [2]:
# A, b, c
A = np.array([[1,0,1,0,1,0,-1,0,0,0],[0,1,0,1,0,1,0,-1,0,0],
              [0,0,2,1,0,0,0,0,1,0],[0,0,1,2,0,0,0,0,0,1]])
c = np.array([35,40,16,19,47,54,0,0,0,0])

In [3]:
Matrix(A)


Matrix([
[1, 0, 1, 0, 1, 0, -1,  0, 0, 0],
[0, 1, 0, 1, 0, 1,  0, -1, 0, 0],
[0, 0, 2, 1, 0, 0,  0,  0, 1, 0],
[0, 0, 1, 2, 0, 0,  0,  0, 0, 1]])

# Pre

In [4]:
def to_polynomial(coef,vars):
            '''
            Function to define a single row of the coefficient as a polynomial
            '''
            res1 = 1
            res2 = 1
            for i in range(len(coef)):
                if coef[i] > 0:
                    res1 = res1*vars[i]**coef[i]
                elif coef[i] < 0:
                    res2 = res2*vars[i]**(-coef[i])
            res = res1 - res2
            return res

In [5]:
def monomial(p):
    return [prod(x**k for x, k in zip(p.gens, mon)) for mon in p.monoms()]

In [6]:
# Define symbolic variables xs for each column (index 0 in Python)
sym_str_x = 'x:' + str(A.shape[1])
xs = symbols(sym_str_x)

# Main

In [7]:
def gen(A):
    # column/row dimension of a matrix
    r = A.shape[0]
    n = A.shape[1]
    A = np.array(A)
    
    # Define symbolic variables xs for each column (index 0 in Python)
    sym_str_x = 'x:' + str(A.shape[1])
    xs = symbols(sym_str_x)
    
    # Calculate a basis for the kernel
    # http://www.numbertheory.org/php/mlll.html
    # the website can fix the gap
    def Ker(A):
        A = Matrix(A)
        B = np.array(Matrix(A).nullspace()).transpose()
        C = Matrix(B[0]).transpose()
        return C
    
    # find an equivalent basis with all base vectors lying in the same orthant
    def same_orthant(A):    
        # compute the highest value in abstract value in a numpy natrix
        def abs_max(A):
            A = np.array(A)
            return abs(max(A.min(), A.max(), key=abs))

        for i in range(1,A.shape[0]):
            A[i,:] = A[i,:]+A[i-1,:]*(1+abs_max(A[i,:]))
        A_fin = np.array(A)
        A_fin = A_fin.astype(int)
        A_fin = Matrix(A_fin)
        return A_fin
    
    # compute J and prepare the function phi & phi_inv
    def nega_index(A):
        J = [ A[A.shape[0]-1,k]<0 for k in range(0,A.shape[1])]
        return J

    def reverse_sign(A):
        n = A.shape[1]
        A = np.array(A)
        for i in range(0,n):
            if nega_index(A)[i]:
                A[:,i] = - A[:,i]
        A_fin = A.astype(int)
        A_fin = Matrix(A_fin)
        return A_fin

    def phi(A):
        G_J=[]
        for k in range(0,A.shape[0]):
            G_J.append(to_polynomial(A[k,:],xs))
        return G_J

    def phi_inv(G_J):
        G_J_len = len(G_J)

        temp = []
        vp = [0]*n
        vm = [0]*n
        for k in range(0,G_J_len):
            for i in range(0,n):
                p = monomial(Poly(G_J[k]))[0]
                m = monomial(Poly(G_J[k]))[1]
                vp[i] = degree(p,xs[i])
                vm[i] = degree(m,xs[i])
            v = np.array(vp) - np.array(vm)
            v = v.astype(int)
            temp.append(v)
            temp_fin = Matrix(temp)
        return temp_fin, temp
    
    
    def TJ_operation(G_J,J):
        for j in J:
            if j:
                # change position of xs[j]
                xs_list = list(xs)
                index = J.index(j)
                J[index] = False
                temp = xs_list[index]
                xs_list.remove(temp)
                xs_list.insert(0,temp)

                G_J = list(groebner(G_J, xs_list , order='lex'))

                A3, _ = phi_inv(G_J)
                A3[:,index] = - A3[:,index]
                G_J = phi(A3)

        return G_J
    
   
    # MAIN
    # generating set for the corresponding matrix kernel
    A1 = Ker(A)
    
    # matrix pre-operation
    A2 = same_orthant(A1)
    J = nega_index(A2)
    A3 = reverse_sign(A2)
    
    # groebner basis T_J operation
    G_J = phi(A3)
    G_zero = TJ_operation(G_J,J)
    return G_zero

In [8]:
def groeb(G_zero,c):
    # column/row dimension of a matrix
    n = c.shape[0]
    
    # Define symbolic variables xs for each column (index 0 in Python)
    sym_str_x = 'x:' + str(A.shape[1])
    xs = symbols(sym_str_x)

    def phi_inv(G_J):
        G_J_len = len(G_J)

        temp = []
        vp = [0]*n
        vm = [0]*n
        for k in range(0,G_J_len):
            for i in range(0,n):
                p = monomial(Poly(G_J[k]))[0]
                m = monomial(Poly(G_J[k]))[1]
                vp[i] = degree(p,xs[i])
                vm[i] = degree(m,xs[i])
            v = np.array(vp) - np.array(vm)
            v = v.astype(int)
            temp.append(v)
            temp_fin = Matrix(temp)
        return temp_fin, temp

    # define new order in Sympy
    class compositeOrder(polys.orderings.MonomialOrder):
        """Composite-Lexicographic order of monomials. """
        alias = 'clex'
        is_global = True

        def __call__(self, monomial):
            return ( np.dot(c,np.array(monomial)),sum(monomial), monomial )
            #return ( monomial )
    polys.orderings.__all__.append('clex')
    clex = compositeOrder()
    polys.orderings._monomial_key['clex'] = compositeOrder() 
    
    # apply groebner basis algorithm 
    G_fin = groebner(G_zero,xs,order='clex')
    _ , G_fin = phi_inv(list(G_fin))
    return G_fin

# Experiment I

In [9]:
def augmentation(z_feas,c,T):
    # z_feaas: feasible point ; c: cost; T: universal test set
    exist_aug = True
    i = 0
    while exist_aug:
        exist_aug = False
        for t in T:
            if np.dot(c, t, out=None)>0 and np. all((z_feas-t>=0)):
                z_feas = z_feas-t
                i = 1+i
                #print('Iteration step', i,': vector', z_feas)
                exist_aug = True
            if np.dot(c, t, out=None)<0 and np. all((z_feas+t>=0)):
                z_feas = z_feas+t
                i = i+1
                #print('Iteration step', i,': vector', z_feas)
                exist_aug = True
    #print('Achieve an optimum!')
    return z_feas

In [10]:
def feasible_solu(b):
    return [0,0,0,0,b[0],b[1],0,0,b[2],b[3]]

In [11]:
start_time = time.time()

T_1 = gen(A)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.12265181541442871 seconds ---


In [12]:
#T_1

In [13]:
T_2 = [xs[1]-xs[5],xs[0]-xs[4],xs[5]*xs[7]-1,xs[4]*xs[6]-1,
       xs[8]*xs[9]**2-xs[3]*xs[7],xs[8]**2*xs[9]-xs[2]*xs[6],
       xs[3]*xs[7]*xs[8]-xs[2]*xs[6]*xs[9],
       xs[3]*xs[4]*xs[8]-xs[2]*xs[5]*xs[9],
       xs[3]**2*xs[4]*xs[7]**2-xs[2]*xs[9]**3,
       xs[2]**2*xs[5]*xs[6]**2-xs[3]*xs[8]**3]

#T_2

In [37]:
start_time = time.time()

opt = []
for i in range(0,600,3):
    b = [300+i,300+i,200+10*i,200+10*i]
    temp = augmentation(feasible_solu(b),c,groeb(T_2,c))
    opt.append(temp)

print("--- %s seconds ---" % (time.time() - start_time))

--- 12.763188123703003 seconds ---


In [16]:
#opt